In [1]:
# Warning control
import warnings
warnings.filterwarnings("ignore")


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"


In [4]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team.",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing "
        "support to {customer}, a super important customer for your company. "
        "You need to make sure that you provide the best support! Make sure to "
        "provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
)


In [5]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support qualituy assurance in your team.",
    backstory=(
        "Your work at crewAI (https://crewai.com) and are now working with yort team on a "
        "request from {customer} ensuring that the support representative is providing the "
        "best support possible. \nYou need to make sure that the support representative " 
        "is providing full complete answers, and make no assumptions."
    ),
    verbose=True,
)


In [6]:
# SerperDevTool -> external service that allows the agent to do search on google
# ScrapeWebsiteTool -> single scrapper that allows the agent to access a given URL and extract contents of that
# WebsiteSearchTool -> RAG over a website ( semantic search over the content of that website )
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool


In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/crews",
)


In [8]:
inquiry_resolytuon = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} if the one that reached out. Make sure to use everything "
        "your know to provide the best support possible. Your must strive to provide a "
        "complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the customer's inquiry that address "
        "all aspects of their question.\n The response should include references "
        "to everything you used to find the answer, including external data or "
        "solutions. Ensure the answer is complete, leaving no questions unanswered, "
        "and maintain a helpful and friendly tone throughout."
    ),
    tools=[docs_scrape_tool], # using interpolation
    agent=support_agent,
)


In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [10]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolytuon, quality_assurance_review],
    verbose=2,
    memory=True, # enables short-tem, long-term, and the entity memories
)


In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI if the one that reached out. Make sure to use everything your know to provide the best support possible. Your must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide detailed and accurate guidance to DeepLearningAI on how to add memory to their crew in order to help them set up and kick off their Crew successfully. I should use the Read website content tool to gather information on how to add memory to a Crew on crewAI.

Action: Read website content
Action Input: {} 

Crews - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearchNavigationCore ConceptsCrewsGet StartedExamplesCr

In [12]:
from IPython.display import Markdown
Markdown(result)



Hello Andrew Ng from DeepLearningAI,

Thank you for reaching out with your inquiry about setting up a Crew and adding memory to it on crewAI. I understand the importance of this task and I am here to provide you with detailed guidance to help you successfully kick off your Crew.

To add memory to your Crew on crewAI, you can follow these steps:

1. Log in to your crewAI account and navigate to the Crew dashboard.
2. Select the specific Crew you want to add memory to.
3. Look for the memory settings or options within the Crew settings.
4. Choose the type of memory you want to add, such as short-term, long-term, or entity memory.
5. Input the required information or data into the memory storage to enhance decision-making and task execution strategies within your Crew.

By adding memory to your Crew, you will be able to store execution memories and improve operational efficiency by enhancing your crew's decision-making capabilities.

If you encounter any difficulties or have further questions during the process, please feel free to reach out to me for additional assistance. I am here to help you every step of the way to ensure a successful setup and kickoff for your Crew on crewAI.

Thank you for choosing crewAI for your AI needs. We are committed to providing you with the best support possible.

Best regards,

[Your Name]
Senior Support Representative
crewAI